In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
!nvidia-smi

Mon Jun 24 12:49:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [8]:
torch.cuda.device_count()

2

In [9]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

In [10]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [11]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [12]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.0403, 0.5224, 0.0155],
        [0.3080, 0.9968, 0.3993]], device='cuda:0')

In [13]:
Z = X.cuda(0)
print(X)
print(Z)
Y + Z

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


tensor([[1.0403, 1.5224, 1.0155],
        [1.3080, 1.9968, 1.3993]], device='cuda:0')

In [14]:
Z.cuda(0) is Z

True

In [15]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net(X)
net[0].weight.data.device

device(type='cuda', index=0)

In [16]:
import time

start_CPU=time.time()
for i in range(1000):
    A = torch.rand(1000, 1000)
    B = torch.rand(1000, 1000)
    C = torch.matmul(A, B)
end_CPU=time.time()

start_GPU=time.time()
for i in range(1000):
    A = torch.rand(1000, 1000, device=try_gpu())
    B = torch.rand(1000, 1000, device=try_gpu())
    C = torch.matmul(A, B)
end_GPU=time.time()

print(f'大计算量任务CPU运算时长: {round((end_CPU - start_CPU)*1000, 2)} ms')
print(f'大计算量任务GPU运算时长: {round((end_GPU - start_GPU)*1000, 2)} ms')

start_CPU=time.time()
A = torch.rand(5, 5)
B = torch.rand(5, 5)
C = torch.matmul(A, B)
end_CPU=time.time()

start_GPU=time.time()
A = torch.rand(5, 5, device=try_gpu())
B = torch.rand(5, 5, device=try_gpu())
C = torch.matmul(A, B)
end_GPU=time.time()

print(f'小计算量任务CPU运算时长: {round((end_CPU - start_CPU) * 1000, 2)} ms')
print(f'小计算量任务CPU运算时长: {round((end_GPU - start_GPU) * 1000, 2)} ms')

大计算量任务CPU运算时长: 22963.41 ms
大计算量任务GPU运算时长: 486.06 ms
小计算量任务CPU运算时长: 2.74 ms
小计算量任务CPU运算时长: 146.89 ms


In [17]:
A = torch.rand(100, 100, device=try_gpu())

start = time.time()
for i in range(1000):
    A = torch.mm(A, A)
    B = torch.norm(A)  # 逐个记录
end = time.time()
print(f'逐个记录耗费时间：{round((end - start) * 1000)} ms')

A = torch.rand(100, 100, device=try_gpu())
start = time.time()
for i in range(1000):
    A = torch.mm(A, A)
B = torch.norm(A)  # 最终记录
end = time.time()
print(f'最终记录耗费时间：{round((end - start) * 1000)} ms')

逐个记录耗费时间：72 ms
最终记录耗费时间：12 ms


In [18]:
A = torch.rand(100, 100, device=try_gpu())
print(A.shape)

torch.Size([100, 100])


In [19]:
a = torch.rand(1000, 1000).to(try_gpu(0))
b = torch.rand(1000, 1000).to(try_gpu(0))
c = torch.rand(1000, 1000).to(try_gpu(1))
d = torch.rand(1000, 1000).to(try_gpu(1))
begintime = time.time()
for i in range(1000):
    e = torch.matmul(a, b)
    f = torch.matmul(c, d)
print(time.time() - begintime)

0.3453059196472168


In [20]:
a = torch.rand(1000, 1000).to(try_gpu(0))
b = torch.rand(1000, 1000).to(try_gpu(0))
c = torch.rand(1000, 1000).to(try_gpu(0))
d = torch.rand(1000, 1000).to(try_gpu(0))
begintime = time.time()
for i in range(1000):
    e = torch.matmul(a, b)
    f = torch.matmul(c, d)
print(time.time() - begintime)

0.9089245796203613
